# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 7 Assignment: Computer Vision Neural Network**

**Student Name: Your Name**

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment Instructions

For this assignment you will use YOLO running on Google CoLab.  It is strongly reccomended to run this assignment on CoLab because the example code below is already setup to get you started with the correct versions of  YOLO.

For this assignment you are provided with 10 image files that contain 10 different webcam pictures taken at the [Venice Sidewalk Cafe](https://www.westland.net/beachcam/) a WebCam that has been in opration since 1996.  You can find the 10 images here:

* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk5.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk6.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk7.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk8.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk9.png
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk10.png

You can see a sample of the WebCam here:

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.png)

YOLO does quite well recognizing objects in this webcam, as the following image illustrates.

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/predictions.jpg)

You are to write a script that counts the number of certian objects in each of the images.  Specificially, you are looking for:

* person
* car
* bicycle
* motorbike
* umbrella
* handbag

Your submit dataframe should also contain a column that identifies which image generated each row.  This column should be named **image** and contain integer numbers between 1 and 10.  There should be 10 rows total.  The complete dataframe should look something like this.

|image|person|car|bicycle|motorbike|umbrella|handbag|
|-|-|-|-|-|-|-|
|1|22|1|2|1|1|0|
|2|33|2|4|1|0|1|
|3|21|0|0|0|3|0|
|...|...|...|...|...|...|...|


The following code simply sets up YOLO and then dumps the classification information for the first image.  It is just meant to get you started.  Read in all 10 images and generate a dataframe that looks something like the following. Use the **submit** function like you did in previoius assiignments.


In [0]:
!git clone https://github.com/thtrieu/darkflow.git
!pip install ./darkflow/

fatal: destination path 'darkflow' already exists and is not an empty directory.
Processing ./darkflow
  Created wheel for darkflow: filename=darkflow-1.0.0-cp36-cp36m-linux_x86_64.whl size=831132 sha256=31ce0f6979a1edfd23bef32b8a9d293aa04c4c098de41820ea6d1119250401fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-84ghijp_/wheels/2f/3a/c5/e84e79d73d5a73aa1b5129a66a40947d9d77a32ebed501e431
Successfully built darkflow


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# The following helper script will create a projects/yolo folder for you 
# and download the needed files.  

!mkdir -p /content/drive/My\ Drive/projects
!mkdir -p /content/drive/My\ Drive/projects/yolo
!mkdir -p /content/drive/My\ Drive/projects/yolo/bin
!mkdir -p /content/drive/My\ Drive/projects/yolo/cfg
!wget https://raw.githubusercontent.com/thtrieu/darkflow/master/cfg/coco.names -O /content/drive/My\ Drive/projects/yolo/cfg/coco.names
!wget https://raw.githubusercontent.com/thtrieu/darkflow/master/cfg/yolo.cfg -O /content/drive/My\ Drive/projects/yolo/cfg/yolo.cfg
!wget https://pjreddie.com/media/files/yolov2.weights -O /content/drive/My\ Drive/projects/yolo/bin/yolo.weights

--2019-08-25 17:05:14--  https://raw.githubusercontent.com/thtrieu/darkflow/master/cfg/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘/content/drive/My Drive/projects/yolo/cfg/coco.names’

/content/drive/My D 100%[===================>]     625  --.-KB/s    in 0s      

2019-08-25 17:05:14 (121 MB/s) - ‘/content/drive/My Drive/projects/yolo/cfg/coco.names’ saved [625/625]

--2019-08-25 17:05:15--  https://raw.githubusercontent.com/thtrieu/darkflow/master/cfg/yolo.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 2

In [1]:
# This is your student key that I emailed to you at the beginnning of the semester.
key = "PPboscDL2M94HCbkbvfOLakXXNy3dh5x2VV1Mlpm"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/assignment_yourname_class7.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class7.ipynb'  # Windows
file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class7.ipynb'  # Mac/Linux

In [0]:
from darkflow.net.build import TFNet
import cv2
import numpy as np
import requests
import os
from scipy import misc
from io import BytesIO
from urllib.request import urlopen

os.chdir('/content/drive/My Drive/projects/yolo') # Google CoLab
#os.chdir('/Users/jheaton/projects/darkflow') # Local

# For GPU (Google CoLab)
options = {"model": "./cfg/yolo.cfg", "load": "./bin/yolo.weights", "threshold": 0.1, "gpu": 1.0}

# For CPU 
#options = {"model": "./cfg/yolo.cfg", "load": "./bin/yolo.weights", "threshold": 0.1}

tfnet = TFNet(options)


# Read image to classify
url = "https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.png"
resp = urlopen(url)
img = np.asarray(bytearray(resp.read()), dtype="uint8")
img = cv2.imdecode(img, cv2.IMREAD_COLOR)

result = tfnet.return_predict(img)
print(result)

Parsing ./cfg/yolo.cfg
Loading ./bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.047690629959106445s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  